In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

In [2]:
vehicle_data = pd.read_csv("vehicle_data.csv.gz",
               compression="gzip")
vehicle_data

,vin,registration_class,zip,model_year,body_type,fuel_type
0,999407G3573,PAS,12866,1976,SUBN,GAS
1,9992313,PAS,13316,1924,CONV,GAS
2,998867,PAS,11501,1952,2DSD,GAS
3,9983692,PAS,10940,1936,2DSD,GAS
4,99782010525,PAS,11225,1978,2DSD,GAS
...,...,...,...,...,...,...
9375975,AAAAAAKE201612240,PAS,14621,1973,2DSD,GAS
9375976,,PAS,14120,1941,4DSD,GAS
9375977,,SRF,10580,1967,2DSD,GAS
9375978,,PAS,14047,1958,CONV,GAS


In [44]:
#sort by zip and type
fuel_df = pd.DataFrame(vehicle_df.groupby(['zip','fuel_type']).size())

#rename columns
fuel_df.rename(columns = {'zip':'Zip_Code', 'fuel_type':'Fuel_Type', 0:'Number_of_Vehicles'}, inplace = True) 

#remove NaN Values
fuel_df = fuel_df.dropna()
fuel_df

Number_of_Vehicles
zip   fuel_type                    
0     DIESEL                      1
      GAS                         2
646   GAS                         5
680   GAS                         1
727   GAS                         1
...                             ...
99821 GAS                         1
99826 GAS                         1
99999 DIESEL                      7
      FLEX                        1
      GAS                       241

[14042 rows x 1 columns]